# The Rubin Scheduler Simulation Archive Data Model

## Visits

**This databases is not intended to store metadata on individual visits, but rather sequences of visits.**

If there are use ever cases where an application is frequently querying small subets of sets of visits, maybe we can consider adding a table of visits. But, for our present use cases, storing the visit metedata in separate files in an S3 bucket is better.

## Visit sequences

- The name "visit sequence" is provisional: I'm looking for a better option.
- "visit sequence" is a table of visit metadata that can be read by `rubin_sim.maf.get_sim_data` or `rubin_scheduler.scheduler.utils.SchemaConverter.opsim2obs`.
    - Documentation I can find is out of date: https://lsst-sims.github.io/sims_ocs/tables/summaryallprops.html
- Presently saved as sqlite3 data files
    - I propose supporting hdf5 files as well (or even instead).
        - More standardized and portable.
        - For `baseline_v4.3_10yrs`, the `sqlite3` file is 719M, local read takes 25s.
        - In `hdf5` (uncompressed), the file is 695M, local read takes 2.2s.
          - `hdf5` has optional built-in compression which shrinks the size of the file at the expense of read and write times. If the download from the S3 bucket is the bottleneck rather than the read into python itself, maybe experimenting with a non-0 compression level would be useful.
        - Whether we do this is not important for the purposes of this metadata database.

## Contents of visit sequence files

Pre-night and progress simulations will typically be completed after pre-loading a set of completed visits.
There is an important question of whether to include the pre-loaded visits in visit sequences placed into the archive, or to limit the rows in the visit table in the archive to newly simulated visits.
Limiting the saved visits to newly simulated visits has several advantages:
1. The pre-night report only uses visits from one night, and this is always one of the newly simulated nights. The pre-night simulation presently simulate three nights, so including the pre-loaded visits would increase the archive space, bandwith used, and time spent downloading data by a factor 1/3 of the nights we are into the survey. For example, less 10 months into the survey, the visit sequence would be 100 times larger than necessary if previous visits are included.
2. Averaged over all timesteps, half of all visits in the simulation used for one time sample in progress reports are completed visits. If the completed visits are retrieved only once, storing only the simulated visits will reduce the size of the visit downloads by a factor of two.
The cost of this efficience is the additional complexity of requiring the client to combine the simulated visits with the parent visits whenever the complete set is needed. This complexity can be hidden in the client, however.

## `simulations` table

Tracks output of opsim simulations, primarily those generated for the pre-night briefing and progress tracking, but baselines might also be useful to include.

| column | type | description |
| --- | --- | --- |
| visitseq_uuid | UUID PRIMARY KEY UNIQUE | RFC 4122 Universally Unique IDentifier (from python's `uuid.uuid4()`)|
| visitseq_sha256 | BYTEA NOT NULL | hash of visit table |
| label | TEXT NOT NULL | label for plots and tables |
| visitseq_url | TEXT | URL of visit sequence (sqlite3, maybe hdf5) |
| telescope | TEXT NOT NULL | "simonyi" or "auxtel" |
| first_day_obs | DATE NOT NULL | day obs of first visit in sequence |
| last_day_obs | DATE NOT NULL | day obs of last visit in sequence |
| tags | JSONB | json array of tags. Recent versions of postgresql support querying inside json arrays |
| creation_time | TIMESTAMP WITH TIME ZONE NOT NULL| when the simulation was run |
| scheduler_version | TEXT | version of `rubin_scheduler` |
| config_repo | TEXT | git repo with configuration |
| config_version | TEXT | revision of git repo with configuration |
| config_script | TEXT | path of script in git repo that provides the configuation |
| config_sha256 | BYTEA | hash of config_script |
| sim_runner_args | JSONB | arguments to sim runner as a json dict |
| rewards_url | TEXT | URL of rewards hdf5 file |
| scheduler_url | TEXT | URL of pickle of scheduler |
| conda_env_sha256 | BYTEA | SHA256 hash of output of `conda list --json` |
| host | TEXT | host on which the simulation was run |
| parent_visitseq_uuid | UUID | UUID of visitseq loaded into scheduler before running |
| parent_last_day_obs | DATE | day_obs of last visit loaded into scheduler before running |

In principle, `config_repo`, `config_version`, and `config_script` should be enough to exactly specify the config file used. However, `config_sha256` is still useful for identifying when the config script did not change across config repository versions, or for identifying config scripts that were not taken from a git repository (in which case the `config_repo` etc. would be `NULL`.)

PostgreSQL has a native json type that can be used in queries, so we can, for example, query for simulations with a given value of `sim_start_mjd` or `n_visit_limit` using the `sim_runner_args` column in this table.

RFC 4122 UUIDs can be generated with the python standard library with `uuid.uuid4()`. These should be generated by whatever process is inserting new data into the table.

SHA-256 is a fast hash function that python can apply to recarrays, and is stable across versions of python (unlike python's `hash`). If we compute the SHA-256 for the `recarray` representation of the visit table, we can detect if we ever fail to reconstruct it exactly. This code fragment shows how this might be computed:
```
import hashlib
visitseq_hash = hashlib.sha256(str(recs.dtype).encode())
visitseq_hash.update(np.ascontiguousarray(recs).data.tobytes())
hex_digest = visitset_hash.hexdigest()
```
To insert it into a BYTEA column in postgresql:
```
sql = f"INSERT INTO obstable (visitshash) VALUES (decode('{hex_higest}', 'hex'))"
```

This would, of course, be handled transparently by the python client.

## `completed` table

Tracks results of visit sequences representing actually compleded visits, primarily (probably entirely) derived from queries to the consdb.

| column | type | description |
| --- | --- | --- |
| visitseq_uuid | UUID PRIMARY KEY | RFC 4122 Universally Unique IDentifier |
| visitseq_sha256 | BYTEA NOT NULL | hash of visit table |
| label | TEXT NOT NULL | label for plots and tables |
| visitseq_url | TEXT | URL of visit sequence (sqlite3, maybe hdf5) |
| telescope | TEXT NOT NULL| "simonyi" or "auxtel" |
| first_day_obs | DATE NOT NULL | day obs of first visit in sequence |
| last_day_obs | DATE NOT NULL | day obs of last visit in sequence |
| tags | JSONB | json array of tags. Recent versions of postgresql support querying inside json arrays |
| creation_time | TIMESTAMP WITH TIME ZONE | when the consdb was queried |
| query | TEXT | The query to the consdb used |

Inclusion of the first and last day obs will let us save incremental updates.

Inclusion of the query will let us select subsets (e.g., just one band), but may not be necessary.

We might sometimes want to create entries in this table with the `visitseq_url` set to `NULL`, if we want to record statistics for a set of visits queried from the consdb and want to record how we got them, but do not need to save the visits themselves.

## `mixedvisitseqs` table

| column | type | description |
| --- | --- | --- |
| visitseq_uuid | UUID PRIMARY KEY | RFC 4122 Universally Unique IDentifier |
| visitseq_sha256 | BYTEA NOT NULL | hash of visit table |
| label | TEXT NOT NULL | label for plots and tables |
| visitseq_url | TEXT | URL of visit sequence (sqlite3, maybe hdf5) |
| telescope | TEXT NOT NULL | "simonyi" or "auxtel" |
| first_day_obs | DATE NOT NULL | day obs of first visit in sequence |
| last_day_obs | DATE NOT NULL | day obs of last visit in sequence |
| tags | JSONB | json array of tags. Recent versions of postgresql support querying inside json arrays |
| last_early_day_obs | DATE | the last day obs drawn from the early parent |
| first_late_day_obs | DATE | the first day obs drawn from the late parent |
| early_parent_uuid | UUID | the UUID of the early parent |
| late_parent_uuid | UUID | the UUID of the late parent |

Note that a client can recover the visit sequence even if the `visitseq_url` column is `NULL` if the early and late parents can be retrieved by querying the early parent for visits between `first_day_obs` and `last_early_day_obs` and the late parent for visits between `first_late_day_obs` and `last_day_obs` and concatenating the results.

I don't know if it's a good idea, but `mixedvisitseqs` visit sequences can in principle be daisy-chained: the parent uuids can themselves refer to other entries in the `mixedvisitseqs` table, allowing for the specification of visit sequences comprised of any number of fragments of other visit sequences. This might be useful as a mechanism for incremental updates to queries of the consdb.

## `visitseq` table

The `simulations`, `completed`, and `mixedvisitseqs` tables all have several fields in common, and we may wish to query a table where we don't have to deal with each separately, for example which joining the parent UUID columns in the `mixedvisitseqs` table to its parents. We could accomplish this with a view, but a better way would be to take advantage of postgresql's inheritence: we can create a parent table with the columns that `simulations`, `completed`, and `mixedvisitseqs` have in common:

| column | type | description |
| --- | --- | --- |
| visitseq_uuid | UUID PRIMARY KEY | RFC 4122 Universally Unique IDentifier |
| visitseq_sha256 | BYTEA NOT NULL | hash of visit table |
| label | TEXT NOT NULL | label for plots and tables |
| visitseq_url | TEXT | URL of visit sequence (sqlite3, maybe hdf5) |
| telescope | TEXT NOT NULL | "simonyi" or "auxtel" |
| first_day_obs | DATE NOT NULL | day obs of first visit in sequence |
| last_day_obs | DATE NOT NULL | day obs of last visit in sequence |
| tags | JSONB | json array of tags. Recent versions of postgresql support querying inside json arrays |

Queries to this table will see rows from all of its children: `simulations`, `completed`, and `mixedvisitseqs`.

## `visitseqstats` table

Statistics of the distributions of visit parameters or values of MAF metrics can be included in the database, for example to make progress plots.

| column | type | description |
| --- | --- | --- |
| visitseq_uuid | UUID REFERENCES visitseq(visiteseq_uuid) | RFC 4122 Universally Unique IDentifier |
| first_dayobs | DATE | The day obs of the first night included in statistics (NULL if all) |
| last_dayobs | DATE | The day obs of the last night included in statistics (NULL if all) |
| band | TEXT | band of visits included or on which metric is measured (NULL if all) |
| name | TEXT | metric or parameter name |
| rubin_sim_version | TEXT | version of rubin_sim used to calculate the metric (NULL if not a rubin_sim metric)|
| count | INTEGER | number of values in distribution |
| mean | DOUBLE PRECISION | mean value of metric |
| std | DOUBLE PRECISION | standard deviation of metric |
| min | DOUBLE PRECISION | min value of metric |
| p05 | DOUBLE PRECISION | 5% quantile of metric distribution |
| q1 | DOUBLE PRECISION | first quartile of metric distribution |
| median | DOUBLE PRECISION | median of metric |
| q3 | DOUBLE PRECISION | third quartile of metric distribution |
| p95 | DOUBLE PRECISION | 95% quantile of metric distribution |
| max | DOUBLE PRECISION | maximum value of metric |

The `first_dayobs`, `last_dayobs`, and `band` columns can be used to indicate a subset of visits included in the statistics.

For example, a pre-night briefing simulation should have rows in this table where `first_dayobs == last_dayobs` and `name == 'slewDistance'` for each night of teh simulation, so a user (e.g. the night synopsis tool) can query the database for the count of visits and rough distribution of slew distances for each night simulated. Other good candidates for visit parameters which might be included are `airmass`, `fiveSigmaDepth`, and `moonDistance`.

MAF metrics can also be stored. For example, the distribution of g band 5-sigma depths of healpixels at dayobs `2028-04-01` might be found in a row with `last_dayobs = '2028-04-01'`, `band = 'g'`, and `name = 'coaddM5 HealpixSlicer'`. These values can be used to make progress plots for the progress reports.

If we wanted even more extensive `maf` support, we could add these columns, but **I don't see a need for this now, and they could be added later if needed**:
| column | type | description |
| --- | --- | --- |
| constraint | TEXT | constraint imposed in maf |
| metric_class_name | TEXT | class name of the metric |
| metric_args | JSONB | arguments to the metric constructor |
| slicer_class_name | TEXT | class name of the slicer |
| slicer_args | JSONB | arguments to the slicer constructor |

These would all be `NULL` for non-MAF parameter statistics.

## `condaenv` (maybe)

While in many cases saving conda environments will probably not be useful, there may be times when it is, and we can save environments in a separate table.
The simplest thing would be to just store the output of `conda list --json` in a table:

| column | type | description |
| --- | --- | --- |
| conda_env_hash | BYTEA PRIMARY KEY | SHA256 hash of output of `conda list --json` |
| conda_env | JSONB NOT NULL | output of `conda list --json` |

The content of the output of `conda list --json` is a list of dictionaries, and postgreSQL would be able to query the contents more efficiently if the keys of these dicts were mapped to columns in a table:

| column | type | description |
| --- | --- | --- |
| conda_env_hash | BYTEA NOT NULL | SHA256 hash of output of `conda list --json` |
| base_url | TEXT NOT NULL |  |
| build_number |  INTEGER NOT NULL | | 
| build_string | TEXT NOT NULL | |
| channel | TEXT NOT NULL | |
| dist_name | TEXT NOT NULL | |
| name | TEXT NOT NULL | |
| platform | TEXT NOT NULL | |
| version | TEXT NOT NULL | |

The first form would be easy to set up and use, and more robust with respect to changes in conda if it ever changed its schema for its output json file.
The second form would support more efficient querying with postgresql and be more closely aligned with database best practices.

We need not require that all environments used be included in this table. For example, saving the detailed environments for the prenight simulations is unlikely to be useful and will take a lot of space, so we should just skip adding these environments to this table.

## Thoughts on presistence

Many of the simulations will be of only transient interest, and will not warrent keeping around for ever. We may, however, want to keep their records in this database around even after the visit sequence data itself has been deleted. We can indicate this by setting the URLs in the various tables to `NULL`.


## Tools

Most of the simulations to be included in this archive will be handled by automated processes, but there should also be `python` and command line APIs to add "hand-generated" simulations or metrics.